# SciFi Movie Analysis

### Importing modules

In [75]:
# importing modules
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
import requests
import os

### Saving Datasets to user computer and loading it dynamically

In [76]:
# saving to paths
path_1 = kagglehub.dataset_download("rajugc/imdb-movies-dataset-based-on-genre")
path_2 = kagglehub.dataset_download("ruchi798/movies-on-netflix-prime-video-hulu-and-disney")
print("Path to dataset files:", path_1)
print("Path to dataset files:", path_2)


Path to dataset files: C:\Users\drvan\.cache\kagglehub\datasets\rajugc\imdb-movies-dataset-based-on-genre\versions\3
Path to dataset files: C:\Users\drvan\.cache\kagglehub\datasets\ruchi798\movies-on-netflix-prime-video-hulu-and-disney\versions\5


In [77]:
# Loading datasets from dynamic path
scifi_file = os.path.join(path_1, "scifi.csv")
stream_file = os.path.join(path_2, "MoviesOnStreamingPlatforms.csv")
if os.path.exists(scifi_file):
    scifi_df = pd.read_csv(scifi_file)
    print("Sci-Fi dataset loaded successfully!")
else:
    print(f"Error: {scifi_file} not found!")
if os.path.exists(stream_file):
    stream_df = pd.read_csv(stream_file)
    print("Stream, dataset loaded successfully!")
else:
    print(f"Error: {stream_file} not found!")

Sci-Fi dataset loaded successfully!
Stream, dataset loaded successfully!


### Cleaning the Movie Lists to prepare merging.

In [83]:
#cleaning scifi df
scifi_df.columns = (
    scifi_df.columns
    .str.upper()
    .str.strip()
    .str.replace('_',' ')
)
scifi_cleaned.columns = scifi_cleaned.columns.str.strip().str.upper()
scifi_cleaned = scifi_df.drop(
    ['MOVIE ID', 'RUNTIME', 'DIRECTOR', 'DIRECTOR ID', 'STAR', 'STAR ID', 'VOTES'], axis='columns'
    )
scifi_cleaned = scifi_cleaned.dropna()

scifi_cleaned['YEAR'] = pd.to_numeric(scifi_cleaned['YEAR'], errors='coerce')
scifi_cleaned['CERTIFICATE'] = scifi_cleaned['CERTIFICATE'].astype('category')

#scifi_cleaned.info()
scifi_cleaned.head()

,MOVIE NAME,YEAR,CERTIFICATE,GENRE,RATING,DESCRIPTION,GROSS(IN $)
6,Black Panther,2018,PG-13,"Action, Adventure, Sci-Fi",7.3,"T'Challa, heir to the hidden but advanced king...",700059566.0
7,Avatar,2009,PG-13,"Action, Adventure, Fantasy",7.9,A paraplegic Marine dispatched to the moon Pan...,760507625.0
11,Interstellar,2014,PG-13,"Adventure, Drama, Sci-Fi",8.6,A team of explorers travel through a wormhole ...,188020017.0
13,The Hunger Games,2012,PG-13,"Action, Adventure, Sci-Fi",7.2,Katniss Everdeen voluntarily takes her younger...,408010692.0
14,Dune,2021,PG-13,"Action, Adventure, Drama",8.0,A noble family becomes embroiled in a war for ...,108327830.0


### Cleaning Streaming Data to prepare for merge.

In [84]:
# Cleaning of stream data
stream_df.columns = (
    stream_df.columns
    .str.upper()
    .str.strip()
    .str.replace('_',' ')
)

stream_cleaned = stream_df.drop(
    ['UNNAMED: 0', 'ID', 'YEAR', 'AGE', 'TYPE'], axis ='columns'
    )
stream_cleaned = stream_cleaned.dropna()

stream_cleaned[['NETFLIX', 'HULU', 'PRIME VIDEO', 'DISNEY+']] = stream_cleaned[['NETFLIX', 'HULU', 'PRIME VIDEO', 'DISNEY+']].astype(bool)
stream_cleaned = stream_cleaned.rename(columns={'TITLE': 'MOVIE NAME'})

#stream_cleaned.info()
stream_cleaned.columns = stream_cleaned.columns.str.strip().str.upper()
stream_cleaned.head()

,MOVIE NAME,ROTTEN TOMATOES,NETFLIX,HULU,PRIME VIDEO,DISNEY+
0,The Irishman,98/100,True,False,False,False
1,Dangal,97/100,True,False,False,False
2,David Attenborough: A Life on Our Planet,95/100,True,False,False,False
3,Lagaan: Once Upon a Time in India,94/100,True,False,False,False
4,Roma,94/100,True,False,False,False


### Sorted columns by rating

In [85]:
#scifi_cleaned
scifi_cleaned_sorted = scifi_cleaned.sort_values(by=['RATING'], ascending=[False])
scifi_cleaned_sorted.head(10)


,MOVIE NAME,YEAR,CERTIFICATE,GENRE,RATING,DESCRIPTION,GROSS(IN $)
18,Inception,2010,PG-13,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,292576195.0
25,The Matrix,1999,R,"Action, Sci-Fi",8.7,When a beautiful stranger leads computer hacke...,171479930.0
149,Star Wars: Episode V - The Empire Strikes Back,1980,PG,"Action, Adventure, Fantasy",8.7,After the Rebels are overpowered by the Empire...,290475067.0
71,Terminator 2: Judgment Day,1991,R,"Action, Sci-Fi",8.6,"A cyborg, identical to the one who failed to k...",204843350.0
11,Interstellar,2014,PG-13,"Adventure, Drama, Sci-Fi",8.6,A team of explorers travel through a wormhole ...,188020017.0
51,Star Wars,1977,PG,"Action, Adventure, Fantasy",8.6,Luke Skywalker joins forces with a Jedi Knight...,322740140.0
47,Alien,1979,R,"Horror, Sci-Fi",8.5,The crew of a commercial spacecraft encounter ...,78900000.0
34,Back to the Future,1985,PG,"Adventure, Comedy, Sci-Fi",8.5,"Marty McFly, a 17-year-old high school student...",210609762.0
35,The Prestige,2006,PG-13,"Drama, Mystery, Sci-Fi",8.5,"After a tragic accident, two stage magicians i...",53089891.0
55,Avengers: Infinity War,2018,PG-13,"Action, Adventure, Sci-Fi",8.4,The Avengers and their allies must be willing ...,678815482.0


In [86]:
scifi_stream_df = scifi_cleaned.merge(
    stream_cleaned[['MOVIE NAME', 'NETFLIX', 'HULU', 'PRIME VIDEO', 'DISNEY+']],
    on='MOVIE NAME',
    how='left'
)
#print(scifi_stream_df.head())
#print(scifi_stream_df.info())
scifi_stream_df


,MOVIE NAME,YEAR,CERTIFICATE,GENRE,RATING,DESCRIPTION,GROSS(IN $),NETFLIX,HULU,PRIME VIDEO,DISNEY+
0,Black Panther,2018,PG-13,"Action, Adventure, Sci-Fi",7.3,"T'Challa, heir to the hidden but advanced king...",700059566.0,False,False,False,True
1,Avatar,2009,PG-13,"Action, Adventure, Fantasy",7.9,A paraplegic Marine dispatched to the moon Pan...,760507625.0,False,False,False,True
2,Interstellar,2014,PG-13,"Adventure, Drama, Sci-Fi",8.6,A team of explorers travel through a wormhole ...,188020017.0,NaN,NaN,NaN,NaN
3,The Hunger Games,2012,PG-13,"Action, Adventure, Sci-Fi",7.2,Katniss Everdeen voluntarily takes her younger...,408010692.0,NaN,NaN,NaN,NaN
4,Dune,2021,PG-13,"Action, Adventure, Drama",8.0,A noble family becomes embroiled in a war for ...,108327830.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1186,Planetfall,2005,Not Rated,"Action, Sci-Fi, Western",2.7,A spaghetti western in science fiction clothin...,2710.0,NaN,NaN,NaN,NaN
1187,Chaos,2002,R,Sci-Fi,3.7,In the near future a young biker leaves his cr...,204274.0,NaN,NaN,NaN,NaN
1188,Jim,2010,Unrated,"Drama, Fantasy, Sci-Fi",5.5,A desperate man hires an advanced biotech firm...,13015.0,NaN,NaN,NaN,NaN
1189,The Strategist Anthology,2013,PG-13,"Action, Adventure, Sci-Fi",6.8,Isaac must stop a madman before he changes the...,1000.0,NaN,NaN,NaN,NaN


In [90]:
scifi_stream_df = scifi_stream_df.dropna()

scifi_stream_df_top_rated = scifi_stream_df.sort_values(by=['RATING'], ascending=[False])
scifi_stream_df_top_rated

,MOVIE NAME,YEAR,CERTIFICATE,GENRE,RATING,DESCRIPTION,GROSS(IN $),NETFLIX,HULU,PRIME VIDEO,DISNEY+
42,Terminator 2: Judgment Day,1991,R,"Action, Sci-Fi",8.6,"A cyborg, identical to the one who failed to k...",204843350.0,True,False,False,False
24,Alien,1979,R,"Horror, Sci-Fi",8.5,The crew of a commercial spacecraft encounter ...,78900000.0,False,False,True,False
30,Avengers: Infinity War,2018,PG-13,"Action, Adventure, Sci-Fi",8.4,The Avengers and their allies must be willing ...,678815482.0,False,False,False,True
7,Avengers: Endgame,2019,PG-13,"Action, Adventure, Drama",8.4,After the devastating events of Avengers: Infi...,858373000.0,False,False,False,True
115,WALL·E,2008,G,"Animation, Adventure, Family",8.4,"In the distant future, a small waste-collectin...",223808164.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
713,Double Dragon,1994,PG-13,"Action, Adventure, Comedy",3.9,Two brothers have half of a powerful ancient C...,2341309.0,False,False,True,False
681,The Avengers,1998,PG-13,"Action, Adventure, Sci-Fi",3.8,Two British Agents team up to stop Sir August ...,23322832.0,False,False,False,True
423,Left Behind,2014,PG-13,"Action, Fantasy, Sci-Fi",3.1,A small group of survivors is left behind afte...,13998282.0,False,False,True,False
526,Battlefield Earth,2000,PG-13,"Action, Adventure, Sci-Fi",2.5,"It's the year 3000 A.D., and the Earth is lost...",21471685.0,True,False,False,False


In [96]:
netflix_count = scifi_stream_df_top_rated[scifi_stream_df_top_rated['NETFLIX'] == True].shape[0]
hulu_count = scifi_stream_df_top_rated[scifi_stream_df_top_rated['HULU'] == True].shape[0]
print(f"NETFLIX ", netflix_count)
print(f"HULU ", hulu_count)

NETFLIX  55
HULU  69


In [126]:
def stream_count(df, column_name):
    count = df[df[column_name] == True].shape[0]
    print(f'Total available on {column_name}: {count}')
    return count

In [127]:
def stream_totals(df):
    stream_services = ['NETFLIX', 'HULU', 'PRIME VIDEO', 'DISNEY+']

    stream_counts = {}
    for service in stream_services:
        count = stream_count(df, service)
        stream_counts[service] = count
    return stream_counts

In [116]:
print(f'Total available on', stream_count(scifi_stream_df_top_rated, 'NETFLIX'), 'NETFLIX')


Total available on 55 NETFLIX


In [128]:
stream_totals(scifi_stream_df_top_rated)

Total available on NETFLIX: 55
Total available on HULU: 69
Total available on PRIME VIDEO: 76
Total available on DISNEY+: 62


{'NETFLIX': 55, 'HULU': 69, 'PRIME VIDEO': 76, 'DISNEY+': 62}